In [1]:
from my_packages import *
from appgeopy import *

In [3]:
disp_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_003\ras2pnt_oChoushui_DISPLACEMENT_LOS_mm.pkl"
disp_df = pd.read_pickle(disp_fpath, compression='zip')
disp_df.head(5)

,PointKey,X_TWD97,Y_TWD97,20220407,20220419,20220501,20220513,20220525,20220606,20220618,...,20240103,20240115,20240127,20240208,20240220,20240303,20240315,20240327,20240408,20240420
0,X237393Y2705479,237393.759851,2.705479e+06,-7.5685,-1.4322,9.5453,16.7358,-9.8940,0.6693,5.8517,...,-10.0947,-6.0932,-17.3372,-4.6029,21.8696,8.5551,1.7418,3.6654,3.9521,16.4717
1,X237382Y2705465,237382.784009,2.705465e+06,-7.2381,-1.5377,9.5850,16.6623,-9.7527,0.2109,5.6738,...,-9.9349,-6.0140,-17.4057,-4.6935,21.8917,8.2449,2.0990,3.7431,4.1485,16.4782
2,X237405Y2705465,237405.139575,2.705465e+06,-7.3403,-1.4823,9.7507,16.5927,-9.7192,0.0266,6.4321,...,-10.1191,-6.2317,-17.4353,-4.4784,21.8770,8.9409,1.4523,3.6148,3.9386,16.6205
3,X237450Y2705438,237450.254591,2.705438e+06,-7.6976,-1.5574,9.6765,16.8299,-10.1043,-0.8461,6.9125,...,-10.5204,-6.2425,-17.9405,-4.3830,21.9045,8.5772,1.8304,2.8947,4.1689,16.3458
4,X237472Y2705438,237472.610164,2.705438e+06,-7.3341,-1.7623,9.9235,16.6388,-9.4748,-0.8454,7.4386,...,-10.6751,-5.8051,-18.4713,-4.1193,22.1453,8.0165,1.8594,2.5556,4.1496,16.3500


In [5]:
disp_geodf = geospatial.convert_to_geodata(df=disp_df, xcoord_col="X_TWD97", ycoord_col="Y_TWD97", crs_epsg="EPSG:3826")

In [6]:
disp_geodf.shape

(3913806, 66)

In [7]:
gps_fpath = r"D:\1000_SCRIPTS\002_PostQE_Scripts\20240527_GPS_Data\20240605\Shapefiles\2023_NewGPS_Locations_TWD97.shp"
gps_geodf = gpd.read_file(gps_fpath)
gps_geodf.head(5)

,STATION,X_TWD97,Y_TWD97,geometry
0,ANES,173423.7707,2.622432e+06,POINT (173423.771 2622431.665)
1,CHSG,177597.8018,2.639739e+06,POINT (177597.802 2639738.601)
2,CHUA,204982.5358,2.662409e+06,POINT (204982.536 2662409.022)
3,DNAN,193697.0268,2.619008e+06,POINT (193697.027 2619007.759)
4,ERLN,190852.6565,2.632728e+06,POINT (190852.656 2632728.166)


In [24]:
cache = []
for idx, row in gps_geodf.iterrows():
    try:
        _temp = geospatial.find_point_neighbors(central_point=row, target_points_gdf=disp_geodf, central_key_column="STATION", buffer_radius=1000)
        cache.append(_temp)
        # print(row.STATION, len(_temp))
    except Exception as e:
        print(e, row.STATION)
        pass

nearby_psc = pd.concat(cache, ignore_index=True)

In [28]:
nearby_psc.loc[:, ["PointKey", "X_TWD97", "Y_TWD97", "STATION"]].reset_index(drop=True).to_pickle(r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PS_surroundGPS2023_PointKey_1000m.pkl", compression='zip')